In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
df = pd.read_excel('2013-2019_TE_Data.xlsx')

In [ ]:
df = df.drop(['Player'], axis=1)
df = df.iloc[np.random.permutation(len(df))]

In [ ]:
train_df = df[:50]
test_df = df[50:]

In [ ]:
train_df['5_x'] = train_df['5_x'].replace(',','',regex=True)
train_df['4_y'] = train_df['4_y'].replace(',','',regex=True)
test_df['5_x'] = test_df['5_x'].replace(',','',regex=True)
test_df['4_y'] = test_df['4_y'].replace(',','',regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
train_df = train_df.astype(float)
test_df = test_df.astype(float)

In [ ]:
train_average_points = train_df.pop('Following_Year_PPG')

test_average_points = test_df.pop('Following_Year_PPG')

In [ ]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((train_df.values, train_average_points.values))
train_dataset = dataset.shuffle(len(df)).batch(1)

dataset = tf.data.Dataset.from_tensor_slices((test_df.values, test_average_points.values))
test_dataset = dataset.shuffle(len(df)).batch(1)

In [ ]:
from tensorflow import keras
def get_compiled_model():
  model = tf.keras.Sequential([
    keras.layers.Flatten(input_shape=(15,)),
    keras.layers.Dense(20, activation='relu'),
	  keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.GaussianNoise(0.1),
    keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=50)

Epoch 1/50
50/50 [==============================] - 0s 1ms/step - loss: 834.8984 - mae: 19.6297
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 40.3854 - mae: 4.9724
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 20.0377 - mae: 3.5456
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 16.0729 - mae: 3.2836
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 18.0603 - mae: 3.5663
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 13.2139 - mae: 3.0195
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 15.3675 - mae: 3.0270
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 10.7828 - mae: 2.5767
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 17.4357 - mae: 3.3349
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 14.3433 - mae: 3.0075
Epoch 11/50
50/50 [==============================] - 0s 1

In [ ]:
test_loss, test_acc = model.evaluate(test_df, test_average_points)
print('Test accuracy:', test_acc)

1/1 [==============================] - 0s 99ms/step - loss: 13.9855 - mae: 2.8058
Test accuracy: 2.805764675140381


In [ ]:
while True:
  model = get_compiled_model()
  model.fit(train_dataset, epochs=50)
  test_loss, test_acc = model.evaluate(test_df, test_average_points)
  print('Test accuracy:', test_acc)
  if test_acc < 2:
    break

In [ ]:
model.save('Newest_Accurate_TE_Model.h5')